In [7]:
# Recargar las bibliotecas necesarias debido al reinicio del estado
import nbformat
import os
from pathlib import Path
import re
from collections import defaultdict
import subprocess

Primero obtenemos el directorio actual de nuestro archivo ipynb

In [8]:
# Obtener el directorio actual del notebook
directorio_actual = Path().resolve()

# Construir la ruta relativa al archivo notebook
notebook_path = os.path.join(directorio_actual, 'proyecto_8.ipynb')

# Verificar si el archivo existe
if not os.path.exists(notebook_path):
    print(f"El archivo {notebook_path} no se encuentra.")
else:
    # Leer el contenido del archivo notebook
    with open(notebook_path, "r", encoding="utf-8") as file_new:
        notebook_content_new = nbformat.read(file_new, as_version=4)
    print("El archivo ha sido leído correctamente.")

El archivo ha sido leído correctamente.


Segundo, detectamos las librerías usadas

In [9]:
def detectar_bibliotecas(notebook_path):
    # Leer el contenido del notebook
    with open(notebook_path, "r", encoding="utf-8") as file:
        notebook = nbformat.read(file, as_version=4)
    
    # Diccionario para almacenar las bibliotecas detectadas
    bibliotecas = defaultdict(int)

    # Expresión regular para detectar importaciones
    patron_import = re.compile(r'^\s*(?:import|from)\s+([^\s.]+)')

    # Recorrer las celdas del notebook
    for cell in notebook.cells:
        if cell.cell_type == 'code':
            # Dividir el contenido de la celda en líneas
            lineas = cell.source.split('\n')
            for linea in lineas:
                # Buscar importaciones en cada línea
                match = patron_import.match(linea)
                if match:
                    biblioteca = match.group(1)
                    bibliotecas[biblioteca] += 1
    
    return dict(bibliotecas)

Tercero, hacemos el documento README.md

In [10]:
# Función para analizar el notebook y generar un README.md
def generar_readme(notebook_path, output_path="../README.md"):
    # Leer el contenido del notebook
    with open(notebook_path, "r", encoding="utf-8") as file:
        notebook = nbformat.read(file, as_version=4)
    
    # Contar las celdas de diferentes tipos
    code_cells = [cell for cell in notebook.cells if cell.cell_type == "code"]
    markdown_cells = [cell for cell in notebook.cells if cell.cell_type == "markdown"]
    
    # Extraer información inicial
    first_markdown = markdown_cells[0].source if markdown_cells else "Sin contenido de markdown"
    first_code = code_cells[0].source if code_cells else "Sin contenido de código"

    # Detectar bibliotecas usadas
    bibliotecas_usadas = detectar_bibliotecas(notebook_path)
    bibliotecas_str = "\n".join([f"- `{biblioteca}`" for biblioteca in bibliotecas_usadas])

    # Crear el contenido del README
    readme_content = f"""# Proyecto 8. Megaline: Smart o Ultra. 

## Descripción
La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tenemos acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos. De este modo realizamos una comparativa de dos modelos: bosque aleatorio y regresión logística, para obtener la mayor exactitud posible, usando como umbral de exactitud:0.75

## Contenido del Notebook
- **Total de celdas:** {len(notebook.cells)}
- **Celdas de markdown:** {len(markdown_cells)}
- **Celdas de código:** {len(code_cells)}

### Procesos de análisis

1. **Carga de datos:**
   - Lectura de datos desde archivos fuente.
2. **Descripción de los datos:**
   - Método info para obtener información general de los datos.
3. **Preprocesamiento de los datos:**
   - Segmentación de los datos en conjuntos de entrenamiento, validación y prueba.
4. **Análisis de los datos:**
   - Analisis de la calidad de los modelos con distintos hiperparámetros.
5. **Conclusiones:**
   - Resumen de los hallazgos y conclusiones del análisis.

## Requisitos del sistema

Para ejecutar este notebook, necesitarás:
- Python 3.8 o superior
- Dependencias listadas en `requirements.txt`

## Ejecución
1. Clona este repositorio:
   ```bash
   git clone <URL-del-repositorio>
   ```

2. Cambia al directorio del proyecto:
   ```bash
   cd <nombre-del-directorio>
   ```

3. Abre y ejecuta el notebook:
   ```bash
   jupyter notebook proyecto_8.ipynb
   ```

## Contribuciones
Si deseas contribuir:
1. Realiza un fork del repositorio.
2. Crea una rama para tus cambios:
   ```bash
   git checkout -b mejora-analisis
   ```
3. Envía un pull request explicando tus cambios.

## Licencia
Consulta el archivo `LICENSE` para más información sobre el uso y distribución de este proyecto.
"""
    with open(output_path, "w", encoding="utf-8") as readme_file:
        readme_file.write(readme_content)
    print(f"README.md generado exitosamente en {output_path}")

Cuarto, hacemos el documento requirements.txt

In [11]:
# Función para obtener la versión de una biblioteca
def obtener_version(biblioteca):
    try:
        # Ejecutar el comando pip show para obtener la versión
        result = subprocess.run(['pip', 'show', biblioteca], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        for line in result.stdout.split('\n'):
            if line.startswith('Version:'):
                return line.split()[-1]
    except Exception as e:
        print(f"No se pudo obtener la versión de {biblioteca}: {e}")
    return None

# Función para generar requirements.txt
def generar_requirements(notebook_path, output_path="../requirements.txt"):
    bibliotecas_usadas = detectar_bibliotecas(notebook_path)
    requirements = []

    for biblioteca in bibliotecas_usadas:
        version = obtener_version(biblioteca)
        if version:
            requirements.append(f"{biblioteca}=={version}")
        else:
            requirements.append(f"{biblioteca}")

    # Escribir el archivo requirements.txt
    with open(output_path, "w", encoding="utf-8") as req_file:
        req_file.write("\n".join(requirements))

    print(f"requirements.txt generado exitosamente en {output_path}")

Quinto, llamamos a la función para hacer ambos documentos

In [12]:
# Llamar a la función generar_readme con la ruta del notebook
generar_readme(notebook_path)

# Llamar a la función generar_requirements con la ruta del notebook
generar_requirements(notebook_path)

README.md generado exitosamente en ../README.md
requirements.txt generado exitosamente en ../requirements.txt
